<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cleaning-Functions" data-toc-modified-id="Cleaning-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cleaning Functions</a></span></li><li><span><a href="#General-Functions" data-toc-modified-id="General-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>General Functions</a></span></li><li><span><a href="#Process-and-Merge-Functions" data-toc-modified-id="Process-and-Merge-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Process and Merge Functions</a></span></li></ul></div>

In [1]:
import import_ipynb
from common_imports import *

importing Jupyter notebook from common_imports.ipynb


## Cleaning Functions

In [2]:
def clean_string(s, patterns_replacements):
    for pattern, replacement in patterns_replacements:
        s = re.sub(pattern, replacement, s)
    return s

def clean_money_columns(df, columns):
    # Define the patterns and replacements for cleaning money strings
    money_patterns_replacements = [('[$.]', ''), ('K', '000'), ('M', '000000')]

    # Clean the specified columns using the clean_string function
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(lambda x: clean_string(x, money_patterns_replacements))
            # Convert to numeric, handling non-numeric values
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            print(f"Column {col} not found in the DataFrame. Skipping.")

    return df


In [3]:
def filter_by_language(df):
    # Check for language columns and filter rows accordingly
    language_cols = [col for col in df.columns if 'language' in col.lower() or 'languages' in col.lower()]
    for col in language_cols:
        if df[col].str.len().max() == 2:
            df = df[df[col] == 'en']
        else:
            df = df[df[col] == 'english']
    
    return df

In [4]:
def convert_column_names(df):
    df.columns = df.columns.str.lower().str.replace(r'[\s]', '_', regex=True)
    return df

In [5]:
titles_with_removed_zeros = set()


def save_titles_with_removed_zeros(output_folder_path):
    with open(os.path.join(output_folder_path, 'titles_with_removed_zeros.txt'), 'w') as f:
        for title in titles_with_removed_zeros:
            f.write(f"{title}\n")
    print("Titles with removed zeros saved as 'titles_with_removed_zeros.txt'.")



def clean_title_columns(df):
    title_cols = [col for col in df.columns if 'title' in col.lower()]
    for col in title_cols:
        if df[col].dtype == object:
            original_titles = df[col].copy()
            df[col] = df[col].str.lower().str.replace('[^\w\s]', '').apply(lambda x: x.lstrip('0') if isinstance(x, str) else x)
            
            # Update the set with titles that had leading zeros removed
            titles_with_removed_zeros.update(df.loc[original_titles != df[col], col].dropna().unique())
    return df



In [6]:
def handle_release_dates(df, date_columns=None):
    if date_columns is None:
        date_columns = ['release_date']
    
    for date_column in date_columns:
        if date_column in df.columns:
            try:
                df[date_column] = pd.to_datetime(df[date_column], errors='coerce', format='%Y-%m-%d')
                df['release_year'] = df[date_column].dt.year.astype(int)   # Converted to integer
                df['release_month'] = df[date_column].dt.month.astype(int) # Converted to integer
                df['release_day'] = df[date_column].dt.day.astype(int)     # Converted to integer
            except Exception as e:
                print(f"Error: '{date_column}' column not in 'YYYY-MM-DD' format. {str(e)}")
    return df


In [7]:
def remove_duplicates(df, columns_to_check):
    if set(columns_to_check).issubset(df.columns):
        duplicates = df[df.duplicated(columns_to_check, keep=False)]
        if not duplicates.empty:
            null_counts = duplicates.isnull().sum(axis=1)
            min_null_count = null_counts.min()
            df = df.drop(duplicates[null_counts > min_null_count].index)
    return df

In [8]:
def handle_date_ranges(df, date_columns=None, year_column=None, start_date=None, end_date=None, start_year=None, end_year=None):
    if date_columns is not None:
        for col in date_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
                if start_date and end_date:
                    df = df[((df[col].isnull()) | ((df[col] >= start_date) & (df[col] <= end_date)))]
    if year_column is not None and start_year is not None and end_year is not None:
        if year_column in df.columns:
            df = df[((df[year_column].isnull()) | ((df[year_column] >= start_year) & (df[year_column] <= end_year)))]
    return df

In [9]:
def convert_string_columns_to_lowercase(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.lower()
    return df


## General Functions

In [10]:
def create_folder(output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

In [11]:
def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
                print(f"{filename} has been deleted")
        except Exception as e:
            print(f"Error: {e}")

In [12]:
def drop_columns_by_keyword_or_name(folder_path, keyword=None, column_names=None, file_name=None):
    if file_name:
        file_paths = [os.path.join(folder_path, file_name)]
    else:
        file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    for file_path in file_paths:
        if not os.path.exists(file_path):
            print(f"{file_path} not found, skipping")
            continue
        df = pd.read_csv(file_path)

        # Drop columns containing the keyword in their names
        if keyword:
            cols_to_drop = [col for col in df.columns if keyword in col]
            df.drop(cols_to_drop, axis=1, inplace=True)

        # Drop the columns with the exact specified names
        if column_names:
            cols_to_drop = [col for col in column_names if col in df.columns]
            df.drop(cols_to_drop, axis=1, inplace=True)

        df.to_csv(file_path, index=False)
        print(f"{os.path.basename(file_path)} updated successfully!")
        
def drop_columns(folder_path, file_name, columns):
    file_path = os.path.join(folder_path, file_name)
    if not os.path.exists(file_path):
        print(f"{file_path} not found, skipping")
        return
    df = pd.read_csv(file_path)
    for col in columns:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)
        else:
            print(f"Column {col} not found in {file_name}, skipping")
    df.to_csv(file_path, index=False)
    print(f"{file_name} updated successfully!")

In [13]:
def drop_rows(file_path, file_patterns_to_columns):
    # Read in the CSV file as a DataFrame
    df = pd.read_csv(file_path, low_memory=False)

    # Apply specific conditions based on the file name
    for pattern, columns in file_patterns_to_columns.items():
        if pattern in file_path:
            if isinstance(columns, list):
                df.dropna(subset=columns, inplace=True)
            elif isinstance(columns, dict):
                df.dropna(subset=columns['subset'], how=columns.get('how', 'any'), inplace=True)

    # Save the DataFrame back to the file
    df.to_csv(file_path, index=False)
    


In [14]:
def remove_duplicates_extra(folder_path=None, file_path=None, column_name=None, delimiter=','):
    if file_path:
        process_file(file_path, column_name, delimiter)
    elif folder_path:
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(folder_path, filename)
                process_file(file_path, column_name, delimiter)

def process_file(file_path, column_name, delimiter):
    # Read in the CSV file as a DataFrame
    df = pd.read_csv(file_path, low_memory=False)

    if column_name and column_name in df.columns:
        # Apply the unique processing to the specified column
        df[column_name] = df[column_name].apply(lambda x: delimiter.join(set(str(x).split(delimiter))))

        # Save the DataFrame back to the CSV file
        df.to_csv(file_path, index=False)
        print(f"Removed duplicates from {column_name} in {file_path}")


In [15]:
def print_duplicates_and_nulls(df, duplicate_column):
    # Identify duplicates in the specified column
    duplicates = df.duplicated(subset=duplicate_column, keep=False)
    duplicate_df = df[duplicates]

    # Print the number of duplicates
    print(f"Number of duplicates in {duplicate_column}: {len(duplicate_df)}")

    # Check for null or zero values in all columns
    for column in duplicate_df.columns:
        null_count = duplicate_df[column].isnull().sum()
        zero_count = (duplicate_df[column] == 0).sum()
        print(f"Number of null values in {column} among duplicates: {null_count}")
        print(f"Number of zero values in {column} among duplicates: {zero_count}")


## Process and Merge Functions

In [16]:
def create_title_key_file(folder_path, output_folder_path, specific_files):
    # Loop over all files in folder
    title_data_set = set()
    title_files = {} # Dictionary to keep track of which files each title appears in

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            # Read in CSV file
            try:
                df = pd.read_csv(os.path.join(folder_path, filename))
            except FileNotFoundError:
                print(f"Error: {filename} not found.")
                continue

            # Convert all string columns to lowercase
            df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

            # Check if file has any columns with 'title' in the name
            title_cols = [col for col in df.columns if 'title' in col.lower()]
            if len(title_cols) == 0:
                continue
                
            # Check for language columns and filter rows accordingly
            language_cols = [col for col in df.columns if 'language' in col.lower() or 'languages' in col.lower()]
            for col in language_cols:
                if df[col].str.len().max() == 2:
                    df = df[df[col] == 'en']
                else:
                    df = df[df[col] == 'english']

            # Check if file has release date columns
            release_date_cols = [col for col in df.columns if 'release_date' in col.lower()]
            has_release_date = len(release_date_cols) > 0
            
            print(f"Duplicate titles in {filename}: {df.duplicated(subset=title_cols).sum()}")


            # Loop through each row in the DataFrame and compare titles
            for _, row in df.iterrows():
                title = ''.join([str(row[col]) for col in title_cols]).strip()

                # Add the file to the list of files for this title
                if title not in title_files:
                    title_files[title] = [filename]
                else:
                    title_files[title].append(filename)
                    
                # Add release date information if available
                if has_release_date:
                    release_dates = [row[col] for col in release_date_cols if pd.notnull(row[col])]
                    if len(release_dates) > 0:
                        release_dates = [datetime.strptime(date.split(' ')[0], '%Y-%m-%d').date() for date in release_dates]
                        release_dates.sort()
                        release_date = release_dates[0]
                        release_year = int(release_date.year) # Ensure integer
                        release_month = int(release_date.month) # Ensure integer
                        release_day = int(release_date.day) # Ensure integer

                       # Create a tuple with the title and release date information
                        title_info_tuple = (title, release_date, release_year, release_month, release_day)

                        # Append the dictionary to the list
                        title_data_set.add(title_info_tuple)

    # Convert the set to a list of dictionaries and then to a DataFrame
    title_data_set = [dict(zip(['title', 'release_date', 'release_year', 'release_month', 'release_day'], tpl)) for tpl in title_data_set]
    title_count_df = pd.DataFrame(title_data_set)

    # Reset the index and add 'title_id' column
    title_count_df.reset_index(drop=True, inplace=True)
    title_count_df['title_id'] = title_count_df.index + 1

    # Save DataFrame to CSV file
    title_count_df.to_csv(os.path.join(output_folder_path, 'title_key.csv'), index=False)
    print(f"Title key file saved as 'title_key.csv'.")
    df = pd.read_csv(os.path.join(output_folder_path, 'title_key.csv'))
    print(df.info())

In [17]:
def add_title_ids_main(input_folder, title_key_path, split_folder, output_folder):
    # Read in the title key CSV file as a DataFrame
    title_key_df = pd.read_csv(os.path.join(title_key_path, 'title_key.csv'))

    # Loop through each file in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.csv') and 'extra' not in file_name:
            file_path = os.path.join(input_folder, file_name)

            # Read in the CSV file as a DataFrame
            df = pd.read_csv(file_path)

            # Get the title columns
            title_cols = [col for col in df.columns if 'title' in col.lower()]

            # Check if the file has any title columns
            if len(title_cols) == 0:
                continue

            # Split the file if it has more than 50,000 rows
            if len(df) > 25000:
                split_files = []
                split_num = 1
                split_size = 25000
                for i in range(0, len(df), split_size):
                    split_file_name = f"{os.path.splitext(file_name)[0]}_split{split_num}.csv"
                    split_num += 1
                    split_file_path = os.path.join(split_folder, split_file_name)
                    split_files.append(split_file_path)
                    df.iloc[i:i+split_size].to_csv(split_file_path, index=False)

                # Loop through each split file
                for split_file_path in split_files:
                    # Read in the CSV file as a DataFrame
                    df_split = pd.read_csv(split_file_path)

                    # Add a new column for the title ID
                    df_split['title_id'] = None

                    # Loop through each row in the DataFrame and add the title ID
                    for i, row in df_split.iterrows():
                        title = ' '.join([str(row[col]) for col in title_cols]).strip()
                        title_id = title_key_df.loc[title_key_df['title'] == title, 'title_id'].values

                        # If a title ID was found, add it to the DataFrame
                        if len(title_id) > 0:
                            df_split.at[i, 'title_id'] = title_id[0]
                        else:
                            # If no title ID was found, drop the row
                            df_split.drop(i, inplace=True)

                    # Save the updated CSV file to the split folder
                    output_file = os.path.join(split_folder, os.path.basename(split_file_path))
                    df_split.to_csv(output_file, index=False)
                    print(f"{split_file_path} cleaned and 'title_id' added. On {output_file}")

                # Concatenate the split files and save to the output_folder
                split_files_df = [pd.read_csv(split_file) for split_file in split_files]
                concat_df = pd.concat(split_files_df)
                output_file = os.path.join(output_folder, file_name)
                concat_df.to_csv(output_file, index=False)
                print(f"{len(split_files)} files concatenated and saved to {output_file}")

            else:
                # Add a new column for the title ID
                df['title_id'] = None

                # Loop through each row in the DataFrame and add the title ID
                for i, row in df.iterrows():
                    title = ' '.join([str(row[col]) for col in title_cols]).strip()
                    title_id = title_key_df.loc[title_key_df['title'] == title, 'title_id'].values
                    # If a title ID was found, add it to the DataFrame
                    if len(title_id) > 0:
                        df.at[i, 'title_id'] = title_id[0]
                    else:
                        # If no title ID was found, drop the row
                        df.drop(i, inplace=True)
                
                # Save the updated CSV file to the output path
                output_file = os.path.join(output_folder, file_name)
                output_file2 = os.path.join(input_folder, file_name)
                df.to_csv(output_file, index=False)
                #df.to_csv(output_file2, index=False)
                print(f"{file_name} cleaned and 'title_id' added.",df.head(2))

In [18]:
def add_title_ids_2(df1_file, df2_file, on_col, folder_path, output_path, compare_path):
    # Load the data from df1
    df1 = pd.read_csv(os.path.join(compare_path, df1_file))

    # Load the data from df2
    df2 = pd.read_csv(os.path.join(folder_path, df2_file))
    
    # Check if on_col exists in df1
    if on_col not in df1.columns:
        raise ValueError(f"{on_col} not found in {df1_file}")
    
    # Check if on_col exists in df2
    if on_col not in df2.columns:
        raise ValueError(f"{on_col} not found in {df2_file}")
    
    # Create a new column for the title_id in df2
    df2['title_id'] = ''
    
    # Keep only the rows in df2 that have a matching value in df1[on_col]
    df2 = df2[df2[on_col].isin(df1[on_col])]
    
    # Loop through each row in df2 and add the corresponding title_id from df1
    for index, row in df2.iterrows():
        title_id = df1.loc[df1[on_col] == row[on_col], 'title_id'].iloc[0]
        df2.loc[index, 'title_id'] = title_id
    
    # Write the updated df2 to a new CSV file in the specified output path
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    output_file = os.path.join(output_path, df2_file)
    df2.to_csv(output_file, index=False)
    print(f"{df2_file} cleaned and 'title_id' added to {output_path}")
    
    # Return the updated df2
    return df2_file

In [19]:
def process_data(input_files, column_names, target_column_name, delimiters, output_folder, output_file, merge_columns, remove_quotes=False, numeric=False, num_files=None):
    # Initialize an empty DataFrame to store the results
    result_df = pd.DataFrame()
    
    # Process the specified number of files or all files if num_files is None
    for file_index, file_path in enumerate(input_files[:num_files]):
        # Read the CSV file
        df = pd.read_csv(file_path)
        print(f"Read file {file_path}:")
        
        # Remove leading and trailing spaces from all string columns
        for col in df.select_dtypes(['object']).columns:
            df[col] = df[col].str.strip()

        # Rename the specific column to the target_column_name
        df.rename(columns={column_names[file_index]: target_column_name}, inplace=True)

        # Get the specified delimiter for this file
        delimiter = delimiters[file_index]

        # Split and explode the column if a delimiter is specified
        if delimiter:
            df[target_column_name] = df[target_column_name].str.split(delimiter)
            df = df.explode(target_column_name)
            print(f"Processed DataFrame for file {file_path}:")

        # Remove single or double quotes if specified
        if remove_quotes:
            df[target_column_name] = df[target_column_name].str.replace('"', '').str.replace("'", "")

        # Concatenate with the result DataFrame
        result_df = pd.concat([result_df, df[['title_id', target_column_name]]])
        print(f"Concatenated result DataFrame after file {file_path}:")

    # Handle missing values
    result_df.dropna(subset=[target_column_name], inplace=True)
    result_df.reset_index(drop=True, inplace=True)

    # Convert to numeric if specified
    if numeric:
        result_df[target_column_name] = result_df[target_column_name].astype(float)

    # Group by the specified column and aggregate title_ids
    unique_df = result_df.groupby(target_column_name)['title_id'].apply(lambda x: ','.join(map(str, x))).reset_index()
    unique_df.columns = [target_column_name, merge_columns]

    # Save to the specified output file
    file_path = os.path.join(output_folder, output_file)
    unique_df.to_csv(file_path, index=False)

    # Add an ID column
    unique_df[f'{target_column_name}_id'] = unique_df.index + 1
    unique_df.to_csv(file_path, index=False)

    print(f"{output_file} saved successfully!")


In [20]:
def remove_duplicates_in_set(title_ids_str):
    # Split the string by commas
    title_ids_list = title_ids_str.split(',')
    
    # Convert the list to a set to remove duplicates
    title_ids_set = set(title_ids_list)
    
    return title_ids_set

def filter_title_ids(title_ids_str, title_id_key_set):
    title_ids_list = title_ids_str[1:-1].replace("'", "").split(', ')
    matching_title_ids = [title_id for title_id in title_ids_list if title_id in title_id_key_set]
    return "{" + ", ".join(matching_title_ids) + "}"

def convert_to_list(title_ids_str):
    return list(ast.literal_eval(title_ids_str))

In [21]:
def drop_duplicate_rows_and_report(folder_path, file_pattern):
    # Iterate through the files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(file_pattern):
            file_path = os.path.join(folder_path, file_name)
            
            # Read in the CSV file as a DataFrame
            df = pd.read_csv(file_path, low_memory=False)

            # Drop any fully duplicate rows
            df.drop_duplicates(inplace=True)

            # Find the counts of duplicated title_id and title values
            duplicate_counts = df['title_id'].duplicated(keep=False).sum()
            
            if 'title' in df.columns:
                duplicate_counts1 = df['title'].duplicated(keep=False).sum()
                print(f"File: {file_name}, Title duplicates: {duplicate_counts1}")
            else: print(f"File: {file_name}, does not have a 'title' column.")
            # You can also remove all but the first occurrence of each duplicate title_id if desired
            # df.drop_duplicates(subset=['title_id'], keep='first', inplace=True)

            # Save the DataFrame back to the file
            df.to_csv(file_path, index=False)
            
            print(f"File: {file_name}, Title_id duplicates: {duplicate_counts}")
            

In [22]:
def merge_and_combine(main_file, combine_columns_mapping, rename_combine_column, output_file):
    # Read only the title_id from the main file
    main_df = pd.read_csv(main_file, usecols=['title_id'])
    
    for file_path, combine_columns in combine_columns_mapping.items():
        input_df = pd.read_csv(file_path, usecols=['title_id'] + combine_columns)

        # Rename the specific columns to the common name
        for column_name in combine_columns:
            input_df.rename(columns={column_name: rename_combine_column}, inplace=True)

        # Merge only on the specified columns
        if rename_combine_column in main_df.columns:
            main_df = main_df.merge(input_df[['title_id', rename_combine_column]], on='title_id')
            main_df[rename_combine_column] = main_df[[rename_combine_column + '_x', rename_combine_column + '_y']].apply(
                lambda row: ','.join(row.dropna().astype(str)), axis=1)
            main_df.drop(columns=[rename_combine_column + '_x', rename_combine_column + '_y'], inplace=True)
        else:
            main_df = pd.merge(main_df, input_df[['title_id', rename_combine_column]], on='title_id')

    main_df.to_csv(output_file, index=False)


In [23]:
def clean_split_date(df, date_column, filename):
    # Split date column into separate columns
    df[['Start_Date', 'End_Date_ft']] = df['date'].str.split('-', expand=True)
    
    # Extract year from filename
    year = str(filename[-8:-4])

    # Clean start date column
    df[['Start_Month', 'Start_Day']] = df['Start_Date'].str.split(' ', expand=True)
    df['Start_Year'] = year
    
    # Clean end date column
    # if month is present in End_Date
    df[['End_Month', 'End_Day']] = df['End_Date_ft'].str.split(' ', expand=True)
    
    # If End_Day is null, then set End_Month == Start_Month and End_Day == End_Date_ft
    if df['End_Day'].isnull().all():
        df['End_Month'] = df['Start_Month']
        df['End_Day'] = df['End_Date_ft']
        df['End_Year'] = year
    else:
        # If End_Day is not null, set End_Day to the corresponding values in End_Date_ft where it is null
        null_mask = df['End_Day'].isnull()
        df.loc[null_mask, 'End_Day'] = df.loc[null_mask, 'End_Date_ft']
        # Set End_Month to Start_Month for the corresponding rows where End_Day was null
        df.loc[null_mask, 'End_Month'] = df.loc[null_mask, 'Start_Month']
        df['End_Year'] = year
    
    # Check if end year should be incremented by 1
    if ((df['Start_Month'] == 'Dec') & (df['End_Month'] == 'Jan')).any():
        # Only update rows where Start_Month is Dec and End_Month is Jan
        mask = (df['Start_Month'] == 'Dec') & (df['End_Month'] == 'Jan')
        df.loc[mask, 'End_Year'] = int(year) + 1
          
    df['End_Year'] = df['End_Year'].astype(str) 
    
    #print('date columns:',df[['End_Month', 'End_Day', 'End_Year', 'Start_Year']].head())
    #print('dec and jan:',((df['Start_Month'] == 'Dec') & (df['End_Month'] == 'Jan')).any())
    
    # Convert date columns to datetime format
    df['Start_Date'] = pd.to_datetime(df['Start_Month'] + ' ' + df['Start_Day'] + ' ' + df['Start_Year'])
    df['End_Date'] = pd.to_datetime(df['End_Month'] + ' ' + df['End_Day'] + ' ' + df['End_Year'], errors='coerce')

    # Drop intermediate columns
    df.drop(columns=['End_Date_ft', 'date'], inplace=True)
    
    return df

In [24]:
def clean_movie_data(folder_path, output_folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join(folder_path, filename))
            except FileNotFoundError:
                print(f"Error: {filename} not found.")
                continue

        # Call the cleaning functions
        df = convert_column_names(df)
        df = clean_title_columns(df)
        df = convert_string_columns_to_lowercase(df)
        
        # Save cleaned DataFrame
        new_filename = filename
        df.to_csv(os.path.join(output_folder_path, new_filename), index=False)
        print(f"{filename} cleaned and saved as {new_filename}.")

In [ ]:
# Function to filter title IDs
def filter_title_ids_2(title_ids_str, valid_title_ids):
    title_ids_set = ast.literal_eval(title_ids_str)
    return str(set(title_id for title_id in title_ids_set if int(title_id) in valid_title_ids))


In [ ]:
def split_movie_files(folder_path, output_folder_path, file_type='csv'):
    # List all files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith(f'.{file_type}')]

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for file in files:
        file_path = os.path.join(folder_path, file)

        # Read the file into a DataFrame
        if file_type == 'csv':
            df = pd.read_csv(file_path)
        elif file_type == 'xlsx':
            df = pd.read_excel(file_path)
        else:
            return 'Unsupported file type'

        # Columns for Movie Revenues, Movie Reviews, and MPAA Ratings
        revenue_columns = ['title_id', 'title', 'domestic_box_office', 'international_box_office', 'worldwide_box_office', 'box_office', 'budget', 'revenue']
        review_columns = ['title_id', 'title', 'user_rating', 'audience_score', 'tomato_meter', 'review', 'review_date', 'comment_count', 'like_count', 'review_text', 'rt_score_sentiment', 'rt_is_top_critic', 'rt_original_score', 'creation_date', 'rating', 'user_rating', 'vote_average', 'vote_count', 'popularity']
        mpaa_columns = ['title_id', 'title', 'mpaa_rating']

        # Splitting the DataFrame
        revenue_df = df[[col for col in df.columns if col in revenue_columns]]
        review_df = df[[col for col in df.columns if col in review_columns]]
        mpaa_df = df[[col for col in df.columns if col in mpaa_columns]]

        # File names for the split files
        base_name = os.path.basename(file_path).split('.')[0]
        revenue_file_name = f'{base_name}_revenue.csv'
        review_file_name = f'{base_name}_reviews.csv'
        # mpaa_file_name = f'{base_name}_mpaa.csv'

        # Full path for the output files
        revenue_file_path = os.path.join(output_folder_path, revenue_file_name)
        review_file_path = os.path.join(output_folder_path, review_file_name)
        mpaa_file_path = '/Users/toniwork/Desktop/Capstone/reference/mpaa.csv'

        # Check and save the DataFrames based on their columns
        if set(revenue_df.columns) not in [{'title'}, {'title_id'}, {'title', 'title_id'}]:
            revenue_df.to_csv(revenue_file_path, index=False)
        else:
            print(f"Skipping {file} for revenue as it contains insufficient columns.")

        if set(review_df.columns) not in [{'title'}, {'title_id'}, {'title', 'title_id'}]:
            review_df.to_csv(review_file_path, index=False)
        else:
            print(f"Skipping {file} for reviews as it contains insufficient columns.")

        if set(mpaa_df.columns) not in [{'title'}, {'title_id'}, {'title', 'title_id'}]:
            mpaa_df.to_csv(mpaa_file_path, index=False)
        else:
            print(f"Skipping {file} for mpaa as it contains insufficient columns.")

    return f'Successfully split the files. Check the output folder: {output_folder_path}'